<a href="https://colab.research.google.com/github/piyush-an/DAMG7245-Fall2023/blob/main/notebooks/Scrape_SEC_forms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install beautifulsoup4

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Define the URL of the SEC Forms webpage
url = 'https://www.sec.gov/forms'

# Send an HTTP GET request to the URL
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table_element = soup.find('table')

In [5]:
rows = table_element.find_all('tr')

In [6]:
data = []

for row in rows[1:]:
  form_no = row.find('span', class_='show-for-small').find_next_sibling(string=True).strip()
  form_title = row.find('td', class_='display-title-content views-field views-field-field-display-title').find('a').get_text()
  form_url = f"https://www.sec.gov{row.find('td', class_='display-title-content views-field views-field-field-display-title').find('a')['href']}"
  last_updated = row.find('time')['datetime']
  response = requests.get(form_url)
  if response.status_code == 200:
    link_valid = 'Y'
  else:
    link_valid = 'N'
  data.append([form_no, form_title, form_url, last_updated, link_valid])

# Create a DataFrame from the list of extracted values
df = pd.DataFrame(data, columns=['Form No', 'Form Title', 'Form URL', 'Last Updated', 'Link Valid'])

# Print the DataFrame
df

,Form No,Form Title,Form URL,Last Updated,Link Valid
0,,Examination Brochure: Information about Examin...,https://www.sec.gov/files/exam-brochure.pdf,2023-01-31,Y
1,,Transferred Federal Employees,https://www.sec.gov/ohr/transferred-federal-em...,2016-05-24,Y
2,,Term Employees,https://www.sec.gov/ohr/term-employees-page-link,2016-05-24,Y
3,1,Application for registration or exemption from...,https://www.sec.gov/files/form1.pdf,1999-02-01,Y
4,1-A,Regulation A Offering Statement (PDF),https://www.sec.gov/files/form1-a.pdf,2021-09-27,Y
...,...,...,...,...,...
162,X-17A-5 Part IIC,"FOCUS Report, Part IIC Instructions (PDF)",https://www.sec.gov/files/formx-17a-5_2c-instr...,2021-10-06,Y
163,X-17A-5 Part IIC,"FOCUS Report, Part IIC (PDF)",https://www.sec.gov/files/formx-17a-5_2c.pdf,2021-10-05,Y
164,X-17A-5 Part III,FOCUS Report Part III (PDF),https://www.sec.gov/files/formx-17a-5_3.pdf,2021-10-05,Y
165,X-17A-5 Schedule I,(Financial and Operational Combined Uniform Si...,https://www.sec.gov/files/formx-17a-5_schedi.pdf,2013-05-01,Y


In [7]:
from google.colab import files
from datetime import datetime
import pytz

est = pytz.timezone('US/Eastern')
current_time_est = datetime.now(est)

timestamp = current_time_est.strftime('%Y-%m-%d_%H-%M-%S')
file_name = f"sec_forms_list_{timestamp}.csv"

df.to_csv(file_name, index=False)

files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>